In [1]:
import numpy as np 
import pickle
import pandas as pd
from coordinate_constructor import create_dataset
from tqdm import tqdm

np.random.seed(42)


In [2]:
ds = create_dataset(100000)

100%|██████████| 100000/100000 [06:43<00:00, 247.57it/s]


In [3]:
out_dic = {'event_time': [], 'flag': [], 'trx_count': [], 'pad_category': [], 'x': [], 'y': []}


for j, obj in tqdm(enumerate(ds)):
    x, y, t = obj[0]
    l = obj[1]

    out_dic['event_time'].append(list(t))
    out_dic['flag'].append(l)
    out_dic['trx_count'].append(len(t))
    out_dic['pad_category'].append([0]*256)
    out_dic['x'].append(list(x / l))
    out_dic['y'].append(list(y / l))

100000it [00:03, 30656.84it/s]


In [4]:
df = pd.DataFrame(out_dic)
df['pendulum_id'] = np.arange(len(df))

In [5]:
df.head()

,event_time,flag,trx_count,pad_category,x,y,pendulum_id
0,"[0.06086283691839688, 0.07202048770225249, 0.2...",2.185431,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.4015762927243535, 0.3344843758469082, -0.45...","[0.9158255735246557, 0.9424012957940499, 0.892...",0
1,"[0.0726108258193494, 0.08414476108936249, 0.16...",3.228931,94,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.9304597803322765, 0.9367802291598812, 0.968...","[-0.36639404632719097, -0.34991827939557607, -...",1
2,"[0.15522971205784214, 0.27044895926332435, 0.4...",3.878920,63,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.8434040490608509, 0.910996391649336, 0.9558...","[-0.537279824698231, -0.4124143237108643, -0.2...",2
3,"[0.19307811219162907, 0.2117723519667045, 0.22...",4.984866,92,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.46897397942180297, 0.5723326709689674, 0.61...","[-0.8832119828361018, -0.820021532486512, -0.7...",3
4,"[0.09009287687924147, 0.21141374143986502, 0.3...",4.722532,73,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.5326887873321751, 0.3976757867093064, 0.291...","[0.8463112050839081, 0.9175260043536283, 0.956...",4


In [6]:
df.loc[:1000].to_parquet('../data/test_hawkes_coordinate_100k.parquet')
df.loc[1000:].to_parquet('../data/train_hawkes_coordinate_100k.parquet')

In [33]:
df['min_y'] = df['y'].apply(min)

In [34]:
np.corrcoef(df['min_y'].values, df['flag'].values)

array([[1.        , 0.05054011],
       [0.05054011, 1.        ]])

In [7]:
mean = df.loc[1000:]['flag'].mean()

In [8]:
((df.loc[:1000]['flag'] - mean)**2).mean()

1.7249891377672095